In [1]:
!pip install schedule

In [2]:
%run ../spark_session.ipynb

In [3]:
import time
import requests
import schedule
from pyspark.sql.functions import explode, col, to_timestamp

In [4]:
def explode_data(df):
    df = df.withColumn("ps", explode(df.ps))
    df = df.withColumn("vs", explode(df.ps.vs))
    return df

In [5]:
def output_formatter(df):
    df = df.withColumnRenamed("hr", "hora") \
                  .withColumn("codigo_parada", col("ps.cp")) \
                  .withColumn("nome_parada", col("ps.np")) \
                  .withColumn("latitude_parada", col("ps.py")) \
                  .withColumn("longitude_parada", col("ps.px")) \
                  .withColumn("prefixo_veiculo", col("ps.vs.p")) \
                  .withColumn("previsao_chegada_parada", col("ps.vs.t")) \
                  .withColumn("flag_acessivel", col("ps.vs.a")) \
                  .withColumn("data_hora", col("ps.vs.ta")) \
                  .withColumn("latitude_veiculo", col("ps.vs.py")) \
                  .withColumn("longitude_veiculo", col("ps.vs.px"))
    
    df = df.withColumn("prefixo_veiculo", explode(col('prefixo_veiculo')))
    df = df.withColumn("previsao_chegada_parada", explode(col('previsao_chegada_parada')))
    df = df.withColumn("flag_acessivel", explode(col('flag_acessivel')))
    df = df.withColumn("data_hora", explode(col('data_hora')))
    df = df.withColumn("latitude_veiculo", explode(col('latitude_veiculo')))
    df = df.withColumn("longitude_veiculo", explode(col('longitude_veiculo')))
    
    df = df.withColumn("data_hora", to_timestamp(col('data_hora')))

    return df.drop("ps","vs") 

In [6]:
def job_raw_preview():
    print("Criando sessão spark e carregando dados /raw/prev")
    spark = get_or_create_session_spark("raw_trusted_prev")
    df = spark.read.json("s3a://raw/prev")
    try:
        print("Aplicando tratamentos")
        df = explode_data(df)
        df = output_formatter(df)
        
        print("Salvando dados!")
        df.write.format("delta").mode("overwrite").save("s3a://trusted/prev")
    except Exception as e:
        print(f"Erro ao transformar os dados \n {e}")

    spark.stop()
    print("Processamento finalizado!")

In [7]:
schedule.every(5).minutes.do(job_raw_preview)

while True:
    schedule.run_pending()
    time.sleep(1)

Criando sessão spark e carregando dados /raw/prev
Obtendo sessão spark raw_trusted_prev
Sessão Spark obtida com sucesso! raw_trusted_prev
Aplicando tratamentos
Salvando dados!


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Erro ao transformar os dados 
 An error occurred while calling o105.save


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
job_raw_preview()